In [ ]:
from sqlalchemy import or_
from flask import Flask, render_template, url_for, request, redirect, session,jsonify, make_response
from flask.helpers import flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user
from flask_bcrypt import Bcrypt
import jwt
import os,sys
import datetime
from functools import wraps

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///todo.db'
app.config['SECRET_KEY'] = 'mysecret'
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

class User(db.Model):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    username = db.Column(db.String(50), unique = True)
    password = db.Column(db.String(80))
    email = db.Column(db.String(255))
    role_id = db.Column(db.Integer, db.ForeignKey('role.id'))

class Role(db.Model):
    __tablename__ = 'role'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    name = db.Column(db.String(50), unique = True)
    description = db.Column(db.String(50))
    users = db.relationship("User", backref = "role")

class Manufacturer(db.Model):
    __tablename__ = 'manufacturer'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    name = db.Column(db.String(200), nullable = False)

class Product(db.Model):
    __tablename__ = 'product'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    name = db.Column(db.String(200), nullable = False)
    category_id = db.Column(db.Integer, db.ForeignKey('category.id'))
    category = db.relationship("Category", backref = "products")
    manufacturer_id = db.Column(db.Integer, db.ForeignKey('manufacturer.id'))
    manufacturer = db.relationship("Manufacturer", backref = "products")
    image = db.Column(db.String(200))
    url = db.Column(db.String(200))
    price = db.Column(db.Float)
    size = db.Column(db.String(20))
    type = db.Column(db.String(20))
    tag = db.Column(db.String(20))
    score = db.Column(db.Float)

class Category(db.Model):
    __tablename__ = 'category'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    name = db.Column(db.String(200), nullable = False)
    friendly_name = db.Column(db.String(200), nullable = False)
    animation_name = db.Column(db.String(200), nullable = False)
    description = db.Column(db.String(200), nullable = False)

class Userrate(db.Model):
    __tablename__ = 'userrate'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    user = db.relationship("User", backref = "rates")
    product_id = db.Column(db.Integer, db.ForeignKey('product.id'))
    product = db.relationship("Product", backref = "rates")
    score = db.Column(db.Integer)
    comment = db.Column(db.String(200))

    
def token_required(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        token = None

        if 'x-access-token' in request.headers:
            token = request.headers['x-access-token']

        if not token:
            return jsonify({'message' : 'Token is missing!'}), 401

        try: 
            data = jwt.decode(token, app.config['SECRET_KEY'])
            current_user = User.query.filter_by(id=data['id']).first()
        except:
            return jsonify({'message' : 'Token is invalid!'}), 401

        return f(current_user, *args, **kwargs)

    return decorated


@app.route('/login')
def login():
    auth = request.authorization

    if not auth or not auth.username or not auth.password:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    user = User.query.filter_by(username=auth.username).first()

    if not user:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    if bcrypt.check_password_hash(user.password, auth.password):
        token = jwt.encode({'id' : user.id, 'exp' : datetime.datetime.utcnow() + datetime.timedelta(minutes=60)}, app.config['SECRET_KEY'])

        return jsonify({'token' : token.decode('UTF-8')})

    return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

@app.route('/userrate', methods=['GET'])
@token_required
def get_all_userrate(current_user):
    userrates = Userrate.query.filter_by(user_id = current_user.id).all()

    output = []

    for userrate in userrates:
        userrate_data = {}
        userrate_data['id'] = userrate.id
        userrate_data['score'] = userrate.score
        userrate_data['comment'] = userrate.comment
        userrate_data['product_id'] = userrate.product_id

        output.append(userrate_data)

    return jsonify({'userrates' : output})

@app.route('/userrate', methods=['POST'])
@token_required
def create_userrate(current_user):
    data = request.get_json()

    product_id = request.form.get("product_id")
    user_id = request.form.get("user_id")
    score = request.form.get("score")
    comment = request.form.get("comment")
    
    new_userrate=Userrate(score = data['score'], comment = data['comment'], product_id = data['product_id'], user_id=current_user.id)
    db.session.add(new_userrate)
    db.session.commit()

    return jsonify({'message' : "User rate created!"})

@app.route('/userrate/<userrate_id>', methods=['PUT'])
@token_required
def update_userrate(current_user, userrate_id):
    userrate = Userrate.query.filter_by(user_id = current_user.id).filter_by(id = userrate_id).first()

    if not userrate:
        return jsonify({'message' : 'No user rate found!'})

    userrate.score = request.form.get("score")
    userrate.comment = request.form.get("comment")
    db.session.commit()

    return jsonify({'message' : 'User rate has been updated!'})

@app.route('/userrate/<userrate_id>', methods=['DELETE'])
@token_required
def delete_userrate(current_user, userrate_id):
    userrate = Userrate.query.filter_by(user_id =current_user.id).filter_by(id = userrate_id).first()

    if not userrate:
        return jsonify({'message' : 'No user rate found!'})

    db.session.delete(userrate)
    db.session.commit()

    return jsonify({'message' : 'User rate deleted!'})


if __name__ == '__main__':
    app.run('localhost', port=9020)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/zhe/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9020/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jan/2022 23:34:22] "PUT /userrate/2 HTTP/1.1" 200 -
